## **Function Calling LLMs - Team Project**

In [16]:
import openai
import json
import os

from dotenv import load_dotenv


load_dotenv()
openai.api_key = os.environ.get("API_KEY")

## Data

## Functions

In [17]:
from registry import Registry
from pprint import pprint

register = Registry()

@register.register
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)


pprint(register.registered_functions)

{'get_current_weather': <function get_current_weather at 0x000001CE0657A200>}


In [18]:
FUNCTIONS = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {
                    "type": "string", 
                    "enum": ["celsius", "fahrenheit"]
                },
            },
            "required": ["location"],
        },
    }
]

## LLM

In [19]:
MODEL_VERSION = "gpt-3.5-turbo-0613" # gpt-4-0613

CONTEXT = {"role": "system", "content": ""}

In [ ]:
def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [CONTEXT, {"role": "user", "content": "{question}"}]
    
    response = openai.ChatCompletion.create(
        model=MODEL_VERSION,
        messages=messages,
        functions=FUNCTIONS,
    )
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function 
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response

print(run_conversation())

In [3]:
def send_message(user_question:str, messages:list) -> dict:
    messages.append({"role": "user", "content": user_question})
    
    response = openai.ChatCompletion.create(
        model=MODEL_VERSION,
        messages=messages,
        functions=FUNCTIONS,
    )
    response_message = response["choices"][0]["message"]
    
    messages.append({"role": "assistant", "content": response_message})    
    return response_message


def handle_function(function:dict) -> json:
    function_name, function_args = function["name"], function["arguments"]
    
    if function_name in register.registered_functions:
        # run function with function_args
        fcn = register[function_name]
        
        from inspect import signature
        
        sig = signature(fcn).bind(**function_args)
        
        # return value
        
        # add context to messages

        # rerun send_message (later)        

def run(user_question:str):    
    messages = [CONTEXT]
    
    # Step 1: send the conversation (context) and available functions to GPT
    response = send_message(user_question, messages)
    
    # Step 2: check if GPT wanted to call a function 
    function = response.get("function_call")
    if function:
        # Step 3: call the function
        fcn_result = handle_function(function)
        
        # Step 4: send the info on the function call and function response to GPT
        response = send_message(fcn_result, messages) # TODO (rewrite send_message to also allow for function_calls to be added not only user_questions)
    
    print(response)

In [ ]:
run()

## Benchmark